## 1.載入套件與環境設置

In [ ]:
pip install transformers datasets scikit-learn accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # 有助除錯 CUDA 問題

import torch
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import numpy as np1
from collections import Counter
from datasets import concatenate_datasets

In [ ]:
!pip install --upgrade datasets
!rm -rf ~/.cache/huggingface/datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


## 2.資料處理

In [ ]:
# 載入中文資料集（只保留正面與負面樣本）
dataset = load_dataset("tyqiangz/multilingual-sentiments", "chinese")
binary_data = dataset["train"].filter(lambda x: x["label"] in [0, 1])

# 檢查數量分布
print(Counter(binary_data["label"]))

# 平衡類別數量
label_counts = Counter(binary_data["label"])
min_count = min(label_counts.values())

label_0 = binary_data.filter(lambda x: x["label"] == 0).select(range(min_count))
label_1 = binary_data.filter(lambda x: x["label"] == 1).select(range(min_count))

balanced_dataset = concatenate_datasets([label_0, label_1]).shuffle(seed=42)

# 快速測試：只取最多 8000 筆資料（實際數量不足則全取）
max_samples = min(8000, len(balanced_dataset))
balanced_dataset = balanced_dataset.select(range(max_samples))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

multilingual-sentiments.py:   0%|          | 0.00/6.23k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/13.5M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/330k [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/120000 [00:00<?, ? examples/s]

Counter({1: 40000, 0: 40000})


Filter:   0%|          | 0/80000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/80000 [00:00<?, ? examples/s]

In [ ]:
# 再分割成訓練與驗證集
train_texts, val_texts, train_labels, val_labels = train_test_split(
    balanced_dataset["text"], balanced_dataset["label"], test_size=0.2, random_state=42, stratify=balanced_dataset["label"]
)

In [ ]:
print("訓練集：", Counter(train_labels))
print("驗證集：", Counter(val_labels))

訓練集： Counter({0: 3211, 1: 3189})
驗證集： Counter({0: 803, 1: 797})


## 3.分詞與資料集準備

In [ ]:
# 載入中文 BERT 分詞器
model_name = "bert-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [ ]:
# 分詞
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

In [ ]:
# 建立 Dataset
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)


## 4.模型載入與訓練

In [ ]:
# 載入模型
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 訓練參數
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,  # 最多保留2個 checkpoint
    save_steps=500,
    eval_steps=500,
    logging_first_step=True
)

# 評估指標
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np1.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="macro"),
    }

# 建立 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 訓練
trainer.train()

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-8-60f354deb4a0>:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
1,0.629700
10,0.731100
20,0.707600
30,0.712100
40,0.675700
50,0.663300
60,0.604900
70,0.575400
80,0.485900
90,0.443500


TrainOutput(global_step=1200, training_loss=0.3514413680632909, metrics={'train_runtime': 1803.038, 'train_samples_per_second': 10.649, 'train_steps_per_second': 0.666, 'total_flos': 5051732262912000.0, 'train_loss': 0.3514413680632909, 'epoch': 3.0})

## 5.模型評估

In [ ]:
# 評估
eval_result = trainer.evaluate()
print("Evaluation result:", eval_result)

Evaluation result: {'eval_loss': 0.46117332577705383, 'eval_accuracy': 0.860625, 'eval_f1': 0.8606236389027236, 'eval_runtime': 36.7338, 'eval_samples_per_second': 43.557, 'eval_steps_per_second': 0.681, 'epoch': 3.0}


## 6.推論功能

In [ ]:
# 推論函數
def predict_sentiment(texts):
    if isinstance(texts, str):
        texts = [texts]

    encodings = tokenizer(texts, truncation=True, padding=True, max_length=512, return_tensors="pt")

    # 把所有 tensor 搬到與模型相同裝置
    device = model.device
    encodings = {k: v.to(device) for k, v in encodings.items()}

    model.eval()
    with torch.no_grad():
        outputs = model(**encodings)

    logits = outputs.logits
    probs = torch.softmax(logits, dim=-1)
    preds = torch.argmax(probs, dim=-1)
    label_map = {0: "正面", 1: "負面"}

    return [
        {
            "text": texts[i],
            "prediction": label_map[preds[i].item()],
            "confidence": round(probs[i][preds[i]].item(), 4)
        }
        for i in range(len(texts))
    ]

## 7.測試案例

In [ ]:

test_texts_simplified = [
    "這家店的服務態度真的非常差。",              # 負面
    "我非常喜歡這部電影，演員很棒！",            # 正面
    "東西還可以，但沒有我期待的那麼好。",        # 中性偏負
    "太失望了，再也不會來了。",                  # 負面
    "這是我吃過最好吃的牛肉麵！",                # 正面
    "店員很親切，讓人感覺很舒服。",              # 正面
    "產品質感普通，有待加強。",                  # 負面或中性
    "整體體驗還不錯，下次會再來。",              # 正面
    "不值得這個價錢。",                          # 負面
    "客服反應迅速，問題很快就解決了。",          # 正面
    "這款手機的續航力真的很弱。",               # 負面
    "功能齊全，但外觀設計不夠吸引人。",          # 中性偏負
    "雖然延遲了一下，但還是準時送達。",          # 中性偏正
    "超出我的預期！真的太棒了。",                # 正面
    "包裝破損，內容物也有刮傷，失望。",          # 負面
    "價格合理，品質也很不錯。",                  # 正面
    "拍照效果不如宣傳所說，差強人意。",          # 負面
    "這次的購物體驗非常愉快。",                  # 正面
    "我不知道該怎麼評價，感覺普普通通。",        # 中性
    "根本就是詐騙，千萬不要買！",                # 負面
]

from pprint import pprint
pprint(predict_sentiment(test_texts_simplified))

[{'confidence': 0.9816, 'prediction': '負面', 'text': '這家店的服務態度真的非常差。'},
 {'confidence': 0.9892, 'prediction': '正面', 'text': '我非常喜歡這部電影，演員很棒！'},
 {'confidence': 0.9789, 'prediction': '負面', 'text': '東西還可以，但沒有我期待的那麼好。'},
 {'confidence': 0.9782, 'prediction': '負面', 'text': '太失望了，再也不會來了。'},
 {'confidence': 0.9895, 'prediction': '正面', 'text': '這是我吃過最好吃的牛肉麵！'},
 {'confidence': 0.9903, 'prediction': '正面', 'text': '店員很親切，讓人感覺很舒服。'},
 {'confidence': 0.981, 'prediction': '負面', 'text': '產品質感普通，有待加強。'},
 {'confidence': 0.9849, 'prediction': '正面', 'text': '整體體驗還不錯，下次會再來。'},
 {'confidence': 0.9817, 'prediction': '負面', 'text': '不值得這個價錢。'},
 {'confidence': 0.9899, 'prediction': '正面', 'text': '客服反應迅速，問題很快就解決了。'},
 {'confidence': 0.9831, 'prediction': '負面', 'text': '這款手機的續航力真的很弱。'},
 {'confidence': 0.9825, 'prediction': '負面', 'text': '功能齊全，但外觀設計不夠吸引人。'},
 {'confidence': 0.9608, 'prediction': '正面', 'text': '雖然延遲了一下，但還是準時送達。'},
 {'confidence': 0.9907, 'prediction': '正面', 'text': '超出我的預期！真的太棒了。'},
 {'confide